### **This notebook creates the test datat sets of positive and negative data**

In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as pltc
import sys
import os
import json

import importlib

#### packages
import helper.strategy_helper as st
import helper.visualization as viz_hp
import helper.helper as hp
import helper.file_helper as file_hp
import config.config as config_hp
import helper.pandas_helper as pd_hp
import helper.twitter_helper as twitter_hp
import helper.slurm_helper as slurm_hp

In [117]:
config = config_hp.config()
poster_path = config['POSTER_PATH']
path = config['PATHS']

dead_tweets_path = poster_path['poster_dead_tweet_file']
external_reply_path = path['external_reply']
splited_reply_ids = path['splited_reply_ids']

#### **Get alive tweets : 34,185**

##### **Get 5 or more replies**

In [3]:
conversation_ids_5 = path['conversation_ids_5']

In [4]:
conversation_ids_5 = file_hp.read_file(conversation_ids_5)

In [5]:
df_external_replies = pd.read_pickle(external_reply_path)

In [6]:
df_external_replies = df_external_replies.astype({
    'poster_tweetid': int
})

df_external_replies = df_external_replies.astype({
    'poster_tweetid': str
})

In [7]:
df_5 = df_external_replies.loc[
    df_external_replies['poster_tweetid'].isin(conversation_ids_5)]

##### **Remove dead tweets**

In [8]:
dead_tweets = file_hp.read_file(dead_tweets_path)

In [9]:
print('Number of dead tweets :', 
      len(dead_tweets))

Number of dead tweets : 61856


In [10]:
df_5['poster_tweetid'].nunique() - len(dead_tweets)

34185

In [11]:
df_5 = df_5.astype({
    'poster_tweetid': int
})

df_5 = df_5.astype({
    'poster_tweetid': str
})

In [12]:
df_alive = df_5.loc[
    ~df_5['poster_tweetid'].isin(dead_tweets)]

In [13]:
df_alive['poster_tweetid'].nunique()

34185

In [14]:
df_alive.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 635261 entries, 973 to 21422171
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   replier_tweetid  635261 non-null  int64  
 1   replier_userid   635261 non-null  object 
 2   poster_tweetid   635261 non-null  object 
 3   poster_userid    635261 non-null  float64
 4   tweet_language   635261 non-null  object 
 5   tweet_text       635261 non-null  object 
 6   tweet_time       635261 non-null  object 
 7   year             635261 non-null  object 
 8   campaign         635261 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 48.5+ MB


##### **Get the whole conversations for the tweets (postive cases)**

In [28]:
splited_reply_ids = path['splited_reply_ids']

In [158]:
extracted_reply_5 = path['extracted_reply_5'] + os.sep + 'job_91_450_455.jsonl'

In [155]:
def parse_tweets(tweet_file, output_file=None):
    '''
    Parse the tweets
    :param tweet_file: Raw tweet file
    :param output_file: File to be saved. If none no file is saved
    
    :return Dataframe
    '''
    all_tweets = []
    total = 0
    with open(tweet_file, 'r') as json_file:
        for row in json_file:
            one_row = json.loads(row)
            print(one_row.keys())
            print(one_row['meta'])
            # print(one_row['__twarc'])
            # print(one_row['includes']['tweets'])
            break
            if 'errors' in one_row:
                for values in one_row['errors']:
                    tweet = set_values_for_tweet_with_error(values)
                    
                    if tweet != None:
                        all_tweets.append(tweet)
            
            if 'data' not in one_row:
                break

In [159]:
# df_all = pd.DataFrame()
poster_alive_tweet_ids = poster_path['poster_alive_tweet_ids']
alive_tweets = file_hp.read_file(poster_alive_tweet_ids)
                                 
for tweet_file in glob.glob(extracted_reply_5):
    print(tweet_file)
    parts = tweet_file.split(os.sep)[-1]
    job_index = parts.split('_')[1]
    file_part = parts.split('.')[0]+'.txt'
    
    # if int(job_index) < 60:
    #     continue
    
    split_ids = splited_reply_ids+os.sep+file_part
    print(split_ids)
    id_file = file_hp.read_file(split_ids)
    print(len(id_file))
    print(id_file)
    parse_tweets(tweet_file) 
    df_tweet = twitter_hp.parse_tweets(tweet_file)
    # print(df_tweet)
    print('Conversations ')
    print(df_tweet['conversation_id'].nunique())
    
    df_conv = df_tweet.loc[df_tweet['conversation_id'].isin(id_file)]
    print(df_conv['conversation_id'].nunique())
    print(set(id_file) - set(df_conv['conversation_id'].unique().tolist()))
    print('-----')
    # break
#     df_all = df_all.append(df_tweet)
    
    break

/N/slate/potem/data/derived/extracted_conversation/extracted_reply_5/job_91_450_455.jsonl
/N/slate/potem/data/derived/conversation/5_reply_count/job_91_450_455.txt
5
['1174602215970897920', '1198210261250953216', '1232644648310595584', '1105831383359021056', '855808001017610240']
dict_keys(['data', 'includes', 'errors', 'meta', '__twarc'])
{'newest_id': '1175105799686905857', 'oldest_id': '1174603626058276864', 'result_count': 34}
Conversations 
4
4
{'1198210261250953216'}
-----


In [112]:
# df_tweet.loc[df_tweet['author_id'].isnull()]

,text,conversation_id,lang,entities,possibly_sensitive,reply_settings,created_at,edit_history_tweet_ids,tweetid,author_id,...,impression_count,expanded_url,display_url,in_reply_to_user_id,referenced_tweets,context_annotations,entity_annotations,cashtags,hashtags,mentions


In [99]:
# python_script='/N/u/potem/Quartz/project/infoOps-strategy/script/py_scripts/merge_jsonl_files.py'
# job_name='combine_all_test_positive'
# slurm_path=config['SLURM_PATH']['slurm_path']

# slurm_hp.creat_and_despatch_job(python_script,
                          #  job_name,
                          #  slurm_path,
                          #  despatch=True,
                          #  logs_path=None
                          # )

In [97]:

def cancel_jobs(start_no, end_no):
    '''
    Cancels the jobs
    :param start_no: starting job no
    :param end_no: ending job no
    '''
    for i in range(start_no, end_no+1):
        print(i)
        command = f'scancel -u potem {i}'

        os.system(command)
# cancel_jobs(1471385, 1471424)

##### **Positive cases: Merge all the tweets to create conversation thread**

In [41]:
test_positive = config['TESTS']['test_path']

In [42]:
df_remaining_conv = pd.read_pickle(test_positive)

In [43]:
df_remaining_conv.columns

Index(['text', 'conversation_id', 'lang', 'entities', 'possibly_sensitive',
       'reply_settings', 'created_at', 'edit_history_tweet_ids', 'tweetid',
       'author_id', 'retweet_count', 'reply_count', 'like_count',
       'quote_count', 'impression_count', 'expanded_url', 'display_url',
       'in_reply_to_user_id', 'referenced_tweets', 'context_annotations',
       'entity_annotations', 'cashtags', 'hashtags', 'mentions'],
      dtype='object')

In [44]:
df_remaining_conv['conversation_id'].nunique()

1293

In [45]:
df_remaining_conv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1339908 entries, 0 to 129
Data columns (total 24 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   text                    1339908 non-null  object 
 1   conversation_id         1339888 non-null  object 
 2   lang                    1339907 non-null  object 
 3   entities                0 non-null        object 
 4   possibly_sensitive      1339907 non-null  object 
 5   reply_settings          1339907 non-null  object 
 6   created_at              1339908 non-null  object 
 7   edit_history_tweet_ids  1339897 non-null  object 
 8   tweetid                 1339908 non-null  object 
 9   author_id               1339907 non-null  object 
 10  retweet_count           1339907 non-null  float64
 11  reply_count             1339907 non-null  float64
 12  like_count              1339907 non-null  float64
 13  quote_count             1339907 non-null  float64
 14  impres

In [46]:
df_alive.columns

Index(['replier_tweetid', 'replier_userid', 'poster_tweetid', 'poster_userid',
       'tweet_language', 'tweet_text', 'tweet_time', 'year', 'campaign'],
      dtype='object')

In [47]:
df_alive_pos = df_alive.merge(df_remaining_conv,
                          left_on='poster_tweetid',
                          right_on='conversation_id',
                         )
df_alive_pos.info()                      

<class 'pandas.core.frame.DataFrame'>
Int64Index: 635261 entries, 973 to 21422171
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   replier_tweetid  635261 non-null  int64  
 1   replier_userid   635261 non-null  object 
 2   poster_tweetid   635261 non-null  object 
 3   poster_userid    635261 non-null  float64
 4   tweet_language   635261 non-null  object 
 5   tweet_text       635261 non-null  object 
 6   tweet_time       635261 non-null  object 
 7   year             635261 non-null  object 
 8   campaign         635261 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 48.5+ MB


In [25]:
# df_alive.['conversation_id']['.nunique()job_81_4000_4050

In [50]:
extracted_reply_5

'/N/slate/potem/data/derived/extracted_conversation/extracted_reply_5/*.jsonl*'

In [91]:
ids_file = splited_reply_ids + os.sep + '*.txt'
poster_dead_tweet_file = poster_path['poster_dead_tweet_file']

dead_tweets = file_hp.read_file(poster_dead_tweet_file)

for tweet_file in glob.glob(ids_file):
    # parts = tweet_file.split(os.sep)[-1]
    # file_part = parts.split('.')[0]+'.txt'
    ids = file_hp.read_file(tweet_file)
    print('File ids ', len(ids))
    print(len(set(ids).intersection(set(dead_tweets))))

    break

File ids  34
34


In [87]:
df_all = pd.DataFrame()
test = []
extracted_reply_5 = path['extracted_reply_5']
for tweet_file in glob.glob(extracted_reply_5+os.sep+'job_81_4000_4050.jsonl'):
    parts = tweet_file.split(os.sep)[-1]
    file_part = parts.split('.')[0]+'.txt'
    ids_file = splited_reply_ids + os.sep + file_part
    ids = file_hp.read_file(ids_file)
    print('File ids ', len(ids))
    
    with open(tweet_file, 'r') as json_file:
        for row in json_file:
            one_row = json.loads(row)
            # print(one_row.keys())
            df_all = df_all.append(
                    twitter_hp.parse_tweets(tweet_file))
            
            for row in one_row['data']:
                # print(row['conversation_id'])
                test.append(row['conversation_id'])
#                 break
            
#             break
#         break


print(len(set(test)))
print(df_all['conversation_id'].nunique())
#     print(tweet_file)
#     parts = tweet_file.split(os.sep)[-1]
#     job_index = parts.split('_')[1]
#     file_part = parts.split('.')[0]+'.txt'
    
#     if int(job_index) <=60:
#         continue
    
#     id_file = file_hp.read_file(
#         splited_reply_ids+os.sep+file_part)
#     print('length of id file ', len(id_file))
#     # print(len(id_file))
        
# # #     print(tweet_file)
  
#     df_tweet = twitter_hp.parse_tweets(tweet_file)
#     print(df_tweet['conversation_id'].nunique())
    
#     df_conv = df_tweet.loc[df_tweet['conversation_id'].isin(id_file)]
#     print(df_conv['conversation_id'].nunique())
#     print(set(id_file) - set(df_conv['conversation_id'].unique().tolist()))
#     print('-----')
#     df_all = df_all.append(df_tweet)
    
    # break
    
poster_dead_tweet_file = poster_path['poster_dead_tweet_file']

dead_tweets = file_hp.read_file(poster_dead_tweet_file)
set(test).intersection(set(dead_tweets))

File ids  24
6
6


/tmp/ipykernel_1523787/1565245604.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = df_all.append(
/tmp/ipykernel_1523787/1565245604.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = df_all.append(
/tmp/ipykernel_1523787/1565245604.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = df_all.append(
/tmp/ipykernel_1523787/1565245604.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = df_all.append(
/tmp/ipykernel_1523787/1565245604.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = df_all.append(


{'856441902069153792',
 '856443224172490752',
 '856608662470758400',
 '857318004211814400',
 '857320225376792576',
 '857337419473051648'}

In [82]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 516 entries, 0 to 85
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   text                    516 non-null    object
 1   conversation_id         516 non-null    object
 2   lang                    516 non-null    object
 3   entities                0 non-null      object
 4   possibly_sensitive      516 non-null    bool  
 5   reply_settings          516 non-null    object
 6   created_at              516 non-null    object
 7   edit_history_tweet_ids  516 non-null    object
 8   tweetid                 516 non-null    object
 9   author_id               516 non-null    object
 10  retweet_count           516 non-null    int64 
 11  reply_count             516 non-null    int64 
 12  like_count              516 non-null    int64 
 13  quote_count             516 non-null    int64 
 14  impression_count        516 non-null    int64 
 15  expande

##### **Negative cases: Get the conversations**

##### **Includes only test cases for now**

In [160]:
control_path = '/N/slate/potem/data/derived/extracted_conversation_old/posters_control'

In [172]:
#Number of tweets that poster have in IO dataset

poster_path = config['POSTER_PATH']
poster_alive_with_tweet_count_file = poster_path['poster_alive_with_tweet_count_file']
ids = file_hp.read_file(poster_alive_with_tweet_count_file)

In [175]:
ids[0]

"[200524435, '2012-07-31 12:57', 2]"

In [173]:
#Control tweets for getting conversations

In [177]:
poster_control = config['POSTER_CONTROL']
poster_control_info = poster_control['poster_control_info']

poster_control_conv = poster_control_info + os.sep +'poster_control_conversation_1.txt'
poster_control_user = poster_control_info + os.sep + 'poster_control_track_1.txt'

In [180]:
poster_user = file_hp.read_file(poster_control_user)

In [186]:
poster_user[0:5]

['200524435', '74448776', '375186723', '337092191', '2674106707']

In [183]:
poster_conv = file_hp.read_file(poster_control_conv)

In [185]:
poster_conv[0:5]

['200524435', '74448776', '375186723', '337092191', '2674106707']

In [ ]:
#Whole conversations 

In [178]:
poster_control_path = config['POSTER_CONTROL']
pc_extracted_conversations = poster_control_path['pc_extracted_conversations']

In [171]:
all_control_conversations = []
all_sum = 0
track = []
for row in ids:
    row = row.strip('][').split(', ')

    user = int(row[0])
    start_time = row[1]
    count = int(row[2])
    
    if count == 0:
        print(user)
    print(count)
    poster_filename = f'control_tweets_{user}.jsonl'
    new_path = os.path.join(poster_control_new_tweets, 
                            poster_filename)
    isExist = os.path.exists(new_path)
    
    if isExist == False:
        continue
        
    df = twitter_hp.parse_tweets(new_path)
    print(df.info())
    break

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   text                    100 non-null    object
 1   conversation_id         100 non-null    object
 2   lang                    100 non-null    object
 3   entities                0 non-null      object
 4   possibly_sensitive      100 non-null    bool  
 5   reply_settings          100 non-null    object
 6   created_at              100 non-null    object
 7   edit_history_tweet_ids  100 non-null    object
 8   tweetid                 100 non-null    object
 9   author_id               100 non-null    object
 10  retweet_count           100 non-null    int64 
 11  reply_count             100 non-null    int64 
 12  like_count              100 non-null    int64 
 13  quote_count             100 non-null    int64 
 14  impression_count        100 non-null    int64 
 15  expande